In [1]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
import warnings
warnings.filterwarnings('ignore')

In [2]:
churnData = pd.read_csv('/Users/elissadejong/lab-cross-validation/files_for_lab/Customer-Churn.csv')
churnData.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No,Yes,No,No,No,No,Month-to-month,29.85,29.85,No
1,Male,0,No,No,34,Yes,Yes,No,Yes,No,No,No,One year,56.95,1889.5,No
2,Male,0,No,No,2,Yes,Yes,Yes,No,No,No,No,Month-to-month,53.85,108.15,Yes
3,Male,0,No,No,45,No,Yes,No,Yes,Yes,No,No,One year,42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,No,No,No,No,No,Month-to-month,70.70,151.65,Yes


In [3]:
cols = []

for i in range(len(churnData.columns)):
    cols.append(churnData.columns[i].lower().replace(' ','_'))

churnData.columns = cols
churnData.head()

,gender,seniorcitizen,partner,dependents,tenure,phoneservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,monthlycharges,totalcharges,churn
0,Female,0,Yes,No,1,No,No,Yes,No,No,No,No,Month-to-month,29.85,29.85,No
1,Male,0,No,No,34,Yes,Yes,No,Yes,No,No,No,One year,56.95,1889.5,No
2,Male,0,No,No,2,Yes,Yes,Yes,No,No,No,No,Month-to-month,53.85,108.15,Yes
3,Male,0,No,No,45,No,Yes,No,Yes,Yes,No,No,One year,42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,No,No,No,No,No,Month-to-month,70.70,151.65,Yes


In [4]:
churnData.dtypes

gender               object
seniorcitizen         int64
partner              object
dependents           object
tenure                int64
phoneservice         object
onlinesecurity       object
onlinebackup         object
deviceprotection     object
techsupport          object
streamingtv          object
streamingmovies      object
contract             object
monthlycharges      float64
totalcharges         object
churn                object
dtype: object

In [5]:
churnData['totalcharges'] = pd.to_numeric(churnData['totalcharges'], errors='coerce').fillna(0).astype(int)

In [6]:
churnData.isna().sum()#/len(churnData)*100

gender              0
seniorcitizen       0
partner             0
dependents          0
tenure              0
phoneservice        0
onlinesecurity      0
onlinebackup        0
deviceprotection    0
techsupport         0
streamingtv         0
streamingmovies     0
contract            0
monthlycharges      0
totalcharges        0
churn               0
dtype: int64

In [7]:
churnData['churn'].value_counts()

No     5174
Yes    1869
Name: churn, dtype: int64

In [8]:
categorical = pd.DataFrame(churnData.select_dtypes(np.object))
numerical = pd.DataFrame(churnData.select_dtypes(np.number))

In [9]:
categorical.head()

,gender,partner,dependents,phoneservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,churn
0,Female,Yes,No,No,No,Yes,No,No,No,No,Month-to-month,No
1,Male,No,No,Yes,Yes,No,Yes,No,No,No,One year,No
2,Male,No,No,Yes,Yes,Yes,No,No,No,No,Month-to-month,Yes
3,Male,No,No,No,Yes,No,Yes,Yes,No,No,One year,No
4,Female,No,No,Yes,No,No,No,No,No,No,Month-to-month,Yes


In [10]:
# drop target variable
categorical = categorical.drop('churn', axis=1)
target = churnData['churn']

In [11]:
# get dummies
categorical = pd.get_dummies(categorical)

In [12]:
churnData_dummies = pd.merge(numerical, categorical, left_index=True, right_index=True)
churnData_dummies.head()

,seniorcitizen,tenure,monthlycharges,totalcharges,gender_Female,gender_Male,partner_No,partner_Yes,dependents_No,dependents_Yes,...,techsupport_Yes,streamingtv_No,streamingtv_No internet service,streamingtv_Yes,streamingmovies_No,streamingmovies_No internet service,streamingmovies_Yes,contract_Month-to-month,contract_One year,contract_Two year
0,0,1,29.85,29,1,0,0,1,1,0,...,0,1,0,0,1,0,0,1,0,0
1,0,34,56.95,1889,0,1,1,0,1,0,...,0,1,0,0,1,0,0,0,1,0
2,0,2,53.85,108,0,1,1,0,1,0,...,0,1,0,0,1,0,0,1,0,0
3,0,45,42.30,1840,0,1,1,0,1,0,...,1,1,0,0,1,0,0,0,1,0
4,0,2,70.70,151,1,0,1,0,1,0,...,0,1,0,0,1,0,0,1,0,0


In [13]:
churnData_dummies_target = pd.merge(churnData_dummies, target, left_index=True, right_index=True)
churnData_dummies_target.head()

,seniorcitizen,tenure,monthlycharges,totalcharges,gender_Female,gender_Male,partner_No,partner_Yes,dependents_No,dependents_Yes,...,streamingtv_No,streamingtv_No internet service,streamingtv_Yes,streamingmovies_No,streamingmovies_No internet service,streamingmovies_Yes,contract_Month-to-month,contract_One year,contract_Two year,churn
0,0,1,29.85,29,1,0,0,1,1,0,...,1,0,0,1,0,0,1,0,0,No
1,0,34,56.95,1889,0,1,1,0,1,0,...,1,0,0,1,0,0,0,1,0,No
2,0,2,53.85,108,0,1,1,0,1,0,...,1,0,0,1,0,0,1,0,0,Yes
3,0,45,42.30,1840,0,1,1,0,1,0,...,1,0,0,1,0,0,0,1,0,No
4,0,2,70.70,151,1,0,1,0,1,0,...,1,0,0,1,0,0,1,0,0,Yes


In [14]:
smote = SMOTE()

In [15]:
X = churnData_dummies_target.drop('churn', axis=1)
y = churnData_dummies_target['churn']

In [16]:
X_sm, y_sm = smote.fit_resample(X, y)
y_sm.value_counts()

Yes    5174
No     5174
Name: churn, dtype: int64

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.20, random_state=42)

In [18]:
transformer = StandardScaler()
transformer.fit(X_train)

StandardScaler()

In [19]:
X_train_trans = transformer.transform(X_train)

In [20]:
X_test_trans = transformer.transform(X_test)

In [21]:
logreg = LogisticRegression()
logreg.fit(X_train_trans, y_train)

LogisticRegression()

In [22]:
print(logreg.score(X_test_trans, y_test))

0.8608695652173913


In [23]:
model = DecisionTreeClassifier()
model.fit(X_train_trans, y_train)
model.score(X_test_trans, y_test)

0.8144927536231884

In [24]:
# tree.plot_tree(model);

In [25]:
model.feature_importances_

array([0.00976749, 0.08439056, 0.17174668, 0.13463314, 0.01132181,
       0.01608017, 0.00991769, 0.01014358, 0.00845136, 0.0066612 ,
       0.00163723, 0.00413638, 0.01520202, 0.03453059, 0.03452029,
       0.01550961, 0.        , 0.01822579, 0.00908593, 0.        ,
       0.00862103, 0.01114741, 0.        , 0.03041905, 0.00416786,
       0.        , 0.00673186, 0.0041999 , 0.        , 0.00559696,
       0.19399217, 0.1062603 , 0.03290191])

In [26]:
tl = TomekLinks('majority')
X_tl, y_tl = tl.fit_resample(X, y)
y_tl.value_counts()

No     4611
Yes    1869
Name: churn, dtype: int64

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X_tl, y_tl, test_size=0.20, random_state=42)

In [28]:
transformer = StandardScaler()
transformer.fit(X_train)

StandardScaler()

In [29]:
X_train_trans = transformer.transform(X_train)

In [30]:
X_test_trans = transformer.transform(X_test)

In [31]:
logreg = LogisticRegression()
logreg.fit(X_train_trans, y_train)

LogisticRegression()

In [32]:
print(logreg.score(X_test_trans, y_test))

0.8078703703703703


In [33]:
model = DecisionTreeClassifier()
model.fit(X_train_trans, y_train)
model.score(X_test_trans, y_test)

0.7469135802469136

In [34]:
# tree.plot_tree(model);

In [35]:
model.feature_importances_

array([0.01775739, 0.12034768, 0.24426709, 0.18881261, 0.01343047,
       0.01665059, 0.0115496 , 0.01160602, 0.00892324, 0.00948779,
       0.00076817, 0.00215124, 0.00868233, 0.        , 0.00877955,
       0.01760681, 0.        , 0.00644357, 0.01222049, 0.        ,
       0.01246209, 0.05253664, 0.00031019, 0.00236684, 0.00470944,
       0.        , 0.00566299, 0.0054904 , 0.        , 0.00848672,
       0.20592056, 0.00193316, 0.00063636])

### Conclusion
In both cases the decision tree classifier delivers less accurate scores. TomekLinks gives lower scores than SMOTE. Hence, further analyses could continue to deeper investigate these and other models in combination with SMOTE. 